In [1]:
#import findspark
#findspark.init()
import pyspark
from pyspark.sql import SparkSession
#spark=SparkSession.builder.getOrCreate()
spark = SparkSession.builder.appName("Spark SQL basic example").getOrCreate()
sc = spark.sparkContext
import matplotlib.pyplot as plt
import pandas
from pyspark.sql.functions import *

In [2]:
spark

df= spark.read.format("csv").option("header", "true").load(r"D:\DBDA\FINAL_PROJECT\v1\vehicles.csv")

In [3]:
df = spark.read.csv(r"vehicles.csv",header=True, inferSchema=True)

In [4]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_url: string (nullable = true)
 |-- price: string (nullable = true)
 |-- year: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- cylinders: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- odometer: string (nullable = true)
 |-- title_status: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- drive: string (nullable = true)
 |-- size: string (nullable = true)
 |-- type: string (nullable = true)
 |-- paint_color: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- description: string (nullable = true)
 |-- county: string (nullable = true)
 |-- state: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- posting_date: string (nu

In [5]:
df.show(5)

+----------+--------------------+--------------------+--------------------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+----+-----+----+----+-----------+---------+-----------+------+-----+----+----+------------+
|        id|                 url|              region|          region_url|price|year|manufacturer|model|condition|cylinders|fuel|odometer|title_status|transmission| VIN|drive|size|type|paint_color|image_url|description|county|state| lat|long|posting_date|
+----------+--------------------+--------------------+--------------------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+----+-----+----+----+-----------+---------+-----------+------+-----+----+----+------------+
|7222695916|https://prescott....|            prescott|https://prescott....| 6000|null|        null| null|     null|     null|null|    null|        null|        null|null| null|null|null|       null|     null|       null|  null|  

In [6]:
df.columns

['id',
 'url',
 'region',
 'region_url',
 'price',
 'year',
 'manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'odometer',
 'title_status',
 'transmission',
 'VIN',
 'drive',
 'size',
 'type',
 'paint_color',
 'image_url',
 'description',
 'county',
 'state',
 'lat',
 'long',
 'posting_date']

In [7]:
from pyspark.sql.functions import col, sum as spark_sum

# Calculate the sum of null values for each column
null_counts = df.select([spark_sum(col(column).isNull().cast('int')).alias(column) for column in df.columns])

# Show the counts
null_counts.show()


+---+----+------+----------+-----+----+------------+-----+---------+---------+-----+--------+------------+------------+------+------+------+------+-----------+---------+-----------+------+-----+-----+-----+------------+
| id| url|region|region_url|price|year|manufacturer|model|condition|cylinders| fuel|odometer|title_status|transmission|   VIN| drive|  size|  type|paint_color|image_url|description|county|state|  lat| long|posting_date|
+---+----+------+----------+-----+----+------------+-----+---------+---------+-----+--------+------------+------------+------+------+------+------+-----------+---------+-----------+------+-----+-----+-----+------------+
|  0|9884|  6901|      6533| 6446|7890|       28937|17506|   187143|   190798|16344|   17801|       21618|       15932|174486|144178|319997|106892|     143839|    13732|      13733|382283|23077|25397|24715|       22477|
+---+----+------+----------+-----+----+------------+-----+---------+---------+-----+--------+------------+------------+-

In [8]:
df.count()

441802

In [10]:
columns_to_delete = ['id', 'url', 'region_url','VIN','image_url','description','county','lat','long','posting_date','size','state']

# Drop the columns
df1 = df.drop(*columns_to_delete)

# Show the DataFrame after dropping columns
#df1.show()

In [11]:
df1.columns

['region',
 'price',
 'year',
 'manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'odometer',
 'title_status',
 'transmission',
 'drive',
 'type',
 'paint_color']

In [12]:
df1.count()

441802

In [13]:
# drop duplicated records :
df2 = df1.distinct()
df2.count()

376299

df3 = df1.dropDuplicates()
df3.count()

In [14]:
from pyspark.sql.functions import col, sum as spark_sum

# Calculate the sum of null values for each column
null_counts = df2.select([spark_sum(col(column).isNull().cast('int')).alias(column) for column in df2.columns])

# Calculate the percentage of null values for each column
total_rows = df2.count()
null_percentages = null_counts.select(
    [((col(column) / total_rows) * 100).alias(column + "_null_percentage") for column in df2.columns]
)

# Show the null percentages
null_percentages.show()


+----------------------+---------------------+--------------------+----------------------------+---------------------+-------------------------+-------------------------+--------------------+------------------------+----------------------------+----------------------------+---------------------+--------------------+---------------------------+
|region_null_percentage|price_null_percentage|year_null_percentage|manufacturer_null_percentage|model_null_percentage|condition_null_percentage|cylinders_null_percentage|fuel_null_percentage|odometer_null_percentage|title_status_null_percentage|transmission_null_percentage|drive_null_percentage|type_null_percentage|paint_color_null_percentage|
+----------------------+---------------------+--------------------+----------------------------+---------------------+-------------------------+-------------------------+--------------------+------------------------+----------------------------+----------------------------+---------------------+------------

We can see that "year", "manufacturer", "model", and "odometer" column has less than 6% of null values. So we can simply drop those rows which have null values in these two columns.

In [15]:
# Drop rows with null values in specified columns
df3 = df2.na.drop(subset=['region','price','year','model','odometer','manufacturer'])

In [16]:
df3.count()

346932

In [17]:
null_count = df3.filter(col("year").isNull()).count()
print("null records" ,null_count)

null_count = df3.filter(col("year").isNotNull()).count()
print("not null records" ,null_count)

total_count = df3.select('year').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df3.groupBy('year').count()
print(value_counts.show(900))

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df3.select("year").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346932
total count  346932
--------------------------------------
+--------------------+-----+
|                year|count|
+--------------------+-----+
| depending on spe...|   28|
|                1953|   55|
|                1957|  117|
|                1987|  393|
|                1956|   98|
|                2016|24145|
|                1936|   21|
|                2020|17024|
|                2012|18901|
|                1958|   49|
|                1943|    1|
|                1915|    1|
|                1972|  298|
|                1931|   26|
|                1988|  410|
|                1926|   10|
|                1938|   14|
|                2019|21961|
| Credit Investiga...|    3|
|                2017|30249|
|                1932|   46|
|                1977|  199|
|                2014|24333|
|                1971|  224|
|                1984|  303|
|                2013|24553|
|                1982|  169|
|                1941|   36|
|  

In [18]:
df3.printSchema()

root
 |-- region: string (nullable = true)
 |-- price: string (nullable = true)
 |-- year: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- cylinders: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- odometer: string (nullable = true)
 |-- title_status: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- drive: string (nullable = true)
 |-- type: string (nullable = true)
 |-- paint_color: string (nullable = true)



For all the missing values in other columns, we have just converted them in to a new category, "unknown"

In [19]:
# Replace null values in all columns with 'unknown'
df4 = df3.fillna('unknown')

In [20]:
# Calculate the sum of null values for each column
null_counts = df4.select([spark_sum(col(column).isNull().cast('int')).alias(column) for column in df4.columns])

# Show the counts
null_counts.show()


+------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+-----+----+-----------+
|region|price|year|manufacturer|model|condition|cylinders|fuel|odometer|title_status|transmission|drive|type|paint_color|
+------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+-----+----+-----------+
|     0|    0|   0|           0|    0|        0|        0|   0|       0|           0|           0|    0|   0|          0|
+------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+-----+----+-----------+



In [21]:
null_count = df4.filter(col("manufacturer").isNull()).count()
print("null records" ,null_count)

null_count = df4.filter(col("manufacturer").isNotNull()).count()
print("not null records" ,null_count)

total_count = df4.select('manufacturer').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df4.groupBy('manufacturer').count()
print(value_counts.show(900))

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df4.select("manufacturer").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346932
total count  346932
--------------------------------------
+--------------------+-----+
|        manufacturer|count|
+--------------------+-----+
|              jaguar| 1813|
|               buick| 4753|
|          mitsubishi| 2874|
|             pontiac| 2022|
| pricing and othe...|   40|
|               lexus| 7138|
|              toyota|29054|
|            chrysler| 5103|
| Optional Credit ...|    3|
|               tesla|  812|
|             lincoln| 3764|
|                audi| 6816|
|              datsun|   61|
|                 bmw|12802|
|                jeep|15913|
|               dodge|11137|
|               rover| 1572|
|             hyundai| 8438|
|                ford|60022|
|          alfa-romeo|  859|
|            cadillac| 5911|
|                 ram|14641|
|               mazda| 4702|
|             ferrari|   69|
|                 kia| 6969|
|       mercedes-benz|10083|
|             porsche| 1179|
|        aston-martin|   22|
|  

In [22]:
from pyspark.sql.functions import when

# Define the list of top 20 manufacturers
manufacturer_values = ['nissan','honda','chevrolet','mercedes-benz','ram','dodge','ford','jeep','toyota','bmw','subaru','volkswagen','kia','cadillac','hyundai','lexus','audi','chrysler','acura','buick']

# Use when function to update the 'manufacturer' column
df5 = df4.withColumn('manufacturer', 
                   when(df4['manufacturer'].isin(manufacturer_values), df4['manufacturer'])
                   .otherwise('others'))

In [23]:
null_count = df5.filter(col("manufacturer").isNull()).count()
print("null records" ,null_count)

null_count = df5.filter(col("manufacturer").isNotNull()).count()
print("not null records" ,null_count)

total_count = df5.select('manufacturer').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df5.groupBy('manufacturer').count()
print(value_counts.show(900))

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df5.select("manufacturer").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346932
total count  346932
--------------------------------------
+-------------+-----+
| manufacturer|count|
+-------------+-----+
|        buick| 4753|
|        lexus| 7138|
|       toyota|29054|
|     chrysler| 5103|
|         audi| 6816|
|       others|46212|
|          bmw|12802|
|         jeep|15913|
|        dodge|11137|
|      hyundai| 8438|
|         ford|60022|
|     cadillac| 5911|
|          ram|14641|
|          kia| 6969|
|mercedes-benz|10083|
|    chevrolet|46111|
|        honda|18088|
|   volkswagen| 8283|
|        acura| 5469|
|       nissan|15796|
|       subaru| 8193|
+-------------+-----+

None
--------------------------------------
+-------------+
| manufacturer|
+-------------+
|        buick|
|        lexus|
|       toyota|
|     chrysler|
|         audi|
|       others|
|          bmw|
|         jeep|
|        dodge|
|      hyundai|
|         ford|
|     cadillac|
|          ram|
|          kia|
|mercedes-benz|
|    chevrolet|
|  

# region :

In [24]:
null_count = df5.filter(col("region").isNull()).count()
print("null records" ,null_count)

null_count = df5.filter(col("region").isNotNull()).count()
print("not null records" ,null_count)

total_count = df5.select('region').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df5.groupBy('region').count()
print(value_counts.show(900))

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df5.select("region").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346932
total count  346932
--------------------------------------
+--------------------+-----+
|              region|count|
+--------------------+-----+
|   central louisiana|  116|
|          western KY|  205|
|           texarkana|  189|
|       daytona beach| 1237|
|   heartland florida|  154|
|  southeast missouri|  176|
|            sandusky|  231|
|        gold country|  689|
|           pensacola|  560|
|        south jersey| 2438|
|       new hampshire| 2217|
|          san marcos|  169|
|           fairbanks|  288|
|     kansas city, MO| 2439|
|          harrisburg|  783|
|       state college|  191|
|              yakima| 1453|
|          moses lake|  816|
|huntsville / decatur| 1072|
|   northern michigan|  557|
|       south florida| 2328|
|saginaw-midland-b...|  821|
|potsdam-canton-ma...|   86|
|        northwest OK|  136|
|             jackson| 1083|
| dallas / fort worth| 2473|
|  mcallen / edinburg|  796|
|         long island| 2555|
|  

+--------------------+
|              region|
+--------------------+
|           texarkana|
|        gold country|
|           fairbanks|
|huntsville / decatur|
|florence / muscle...|
|             modesto|
|     imperial county|
|       inland empire|
|              mobile|
|               chico|
|     kenai peninsula|
|           jonesboro|
|    southeast alaska|
|    mendocino county|
|            show low|
|         little rock|
|          tuscaloosa|
|    hanford-corcoran|
|          fort smith|
|  flagstaff / sedona|
+--------------------+
only showing top 20 rows

416


In [25]:
from pyspark.sql.functions import when

# Count the occurrences of each manufacturer value
manufacturer_counts = df5.groupBy('region').count()

# Sort the counts in descending order and select the top 20 manufacturers
top_manufacturers = manufacturer_counts.orderBy('count', ascending=False).limit(50)

# Extract the top 20 manufacturer values
manufacturer_values = [row['region'] for row in top_manufacturers.collect()]

# Use when function to update the 'manufacturer' column
df6 = df5.withColumn('region', 
                     when(df5['region'].isin(manufacturer_values), df5['region'])
                     .otherwise('others'))


In [26]:
null_count = df6.filter(col("region").isNull()).count()
print("null records" ,null_count)

null_count = df6.filter(col("region").isNotNull()).count()
print("not null records" ,null_count)

total_count = df6.select('region').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df6.groupBy('region').count()
print(value_counts.show(900))

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df6.select("region").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346932
total count  346932
--------------------------------------
+--------------------+------+
|              region| count|
+--------------------+------+
|        south jersey|  2438|
|     kansas city, MO|  2439|
| dallas / fort worth|  2473|
|         long island|  2555|
|             atlanta|  2460|
|        jacksonville|  2636|
|             chicago|  2627|
|      seattle-tacoma|  2758|
|              denver|  2698|
|       inland empire|  2345|
|      tampa bay area|  2495|
|            stockton|  2554|
|           charlotte|  2383|
|          cincinnati|  2473|
|       orange county|  2546|
|        st louis, MO|  2469|
|    colorado springs|  2420|
|               boise|  2329|
|              boston|  2617|
|          pittsburgh|  2490|
|             orlando|  2471|
|           milwaukee|  2446|
|        grand rapids|  2524|
|              others|221186|
|      washington, DC|  2548|
|       oklahoma city|  2344|
|           las vegas|  2382|
| 

# model :

In [27]:
null_count = df6.filter(col("model").isNull()).count()
print("null records" ,null_count)

null_count = df6.filter(col("model").isNotNull()).count()
print("not null records" ,null_count)

total_count = df6.select('model').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df6.groupBy('model').count()
print(value_counts.show(900))

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df6.select("model").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346932
total count  346932
--------------------------------------
+--------------------+-----+
|               model|count|
+--------------------+-----+
|             4runner|  903|
|                 hse|   55|
|       sonata hybrid|   73|
|                hr-v|  146|
|compass fwd dr sport|    2|
|                 dts|  239|
|            lacrosse|  358|
|          crv lx awd|   22|
|         verano base|   11|
|transit connect c...|   26|
|           sentra sv|  155|
|  cherokee trailhawk|   25|
|journey r/t flexible|    2|
|           pruis two|    2|
|silverado 2500hd ...|    5|
|                 xk8|   25|
|          charger se|   59|
|journey se sport ...|   34|
|            aerostar|    5|
|      avalon limited|   60|
|           cts sedan|   46|
|                 q40|    8|
|  f 250 platinum fx4|    4|
|      f-150 long bed|    1|
|               azera|   50|
|silverado durmax ...|   12|
|        slt crew cab|    4|
|explorer limited 4wd|   14|
|xc

+--------------------+
|               model|
+--------------------+
|            lacrosse|
|             4runner|
|                 dts|
|           cts sedan|
|                 hse|
|super duty f-350 ...|
|                 xk8|
|                hr-v|
|       sonata hybrid|
|          charger se|
|      avalon limited|
|         2500 diesel|
|               gem 2|
|               azera|
|transit connect c...|
|  cherokee trailhawk|
|   eclipse cross sel|
|           sentra sv|
|          crv lx awd|
|                m 37|
+--------------------+
only showing top 20 rows

23551


In [28]:
from pyspark.sql.functions import when

# Count the occurrences of each model value
manufacturer_counts = df6.groupBy('model').count()

# Sort the counts in descending order and select the top 50 model
top_manufacturers = manufacturer_counts.orderBy('count', ascending=False).limit(50)

# Extract the top 50 model values
manufacturer_values = [row['model'] for row in top_manufacturers.collect()]

# Use when function to update the 'model' column
df7 = df6.withColumn('model',
                     when(df6['model'].isin(manufacturer_values), df6['model'])
                     .otherwise('others'))


In [29]:
null_count = df7.filter(col("model").isNull()).count()
print("null records" ,null_count)

null_count = df7.filter(col("model").isNotNull()).count()
print("not null records" ,null_count)

total_count = df7.select('model').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df7.groupBy('model').count()
print(value_counts.show(900))

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df7.select("model").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346932
total count  346932
--------------------------------------
+------------------+------+
|             model| count|
+------------------+------+
|              1500|  3345|
|            sentra|  1059|
|  silverado 2500hd|  1302|
|            altima|  2159|
|            sierra|  1046|
|     grand caravan|  1340|
|            camaro|  1015|
|          forester|  1120|
|            fusion|  1556|
|             prius|  1219|
|         silverado|  2502|
|             jetta|  1188|
|            tacoma|  2050|
|            tundra|  1177|
|          suburban|   925|
|            sonata|  1326|
|             tahoe|  1406|
|              2500|  2243|
|            others|262576|
|           corolla|  1765|
|             focus|  1482|
|       sierra 1500|  1335|
|              cr-v|  1595|
|          wrangler|  2155|
|            sienna|   908|
|            escape|  2244|
|             rogue|  1045|
|              3500|  1278|
|             cruze|  1245|
|     

In [30]:
from pyspark.sql.functions import countDistinct

# Iterate over each column in the DataFrame
for col_name in df7.columns:
    # Count the distinct values in the column
    distinct_count = df7.select(countDistinct(col_name)).collect()[0][0]
    print(col_name, distinct_count)

region 51
price 15286
year 119
manufacturer 21
model 51
condition 58
cylinders 33
fuel 17
odometer 101992
title_status 19
transmission 82
drive 12
type 22
paint_color 49


In [31]:
null_count = df7.filter(col("transmission").isNull()).count()
print("null records" ,null_count)

null_count = df7.filter(col("transmission").isNotNull()).count()
print("not null records" ,null_count)

total_count = df7.select('transmission').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df7.groupBy('transmission').count()
print(value_counts.show(900))

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df7.select("transmission").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346932
total count  346932
--------------------------------------
+--------------------+------+
|        transmission| count|
+--------------------+------+
|             unknown|  1327|
| and smog fees   ...|     1|
|           automatic|267924|
|               other| 58572|
|2021-05-01T11:30:...|     1|
| and smog fees   ...|     1|
| and smog fees   ...|     1|
|         -122.332289|     3|
|              manual| 19017|
| and smog fees   ...|     1|
| and smog fees   ...|     1|
|2021-05-02T13:04:...|     1|
|2021-05-02T14:34:...|     1|
| and smog fees   ...|     1|
|2021-04-27T14:34:...|     1|
|2021-05-03T15:20:...|     1|
|2021-05-01T13:05:...|     1|
| and smog fees   ...|     1|
|2021-04-27T15:19:...|     1|
| Si se encuentra ...|     1|
| and smog fees   ...|     1|
|           Extended |     1|
|          -73.900821|    12|
|2021-05-02T15:19:...|     1|
| and smog fees   ...|     1|
|2021-05-01T15:19:...|     1|
|2021-04-27T09:58:...|     1|
|2

In [32]:
#Define a list of values to match
names_to_match = ['automatic','manual','other','unknown']
 
#Filter DataFrame to select rows where name is in the list of values or is null
df8 = df7.filter((col("transmission").isin(names_to_match)) )

In [33]:
null_count = df8.filter(col("transmission").isNull()).count()
print("null records" ,null_count)

null_count = df8.filter(col("transmission").isNotNull()).count()
print("not null records" ,null_count)

total_count = df8.select('transmission').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df8.groupBy('transmission').count()
print(value_counts.show(900))

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df8.select("transmission").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346840
total count  346840
--------------------------------------
+------------+------+
|transmission| count|
+------------+------+
|     unknown|  1327|
|   automatic|267924|
|       other| 58572|
|      manual| 19017|
+------------+------+

None
--------------------------------------
+------------+
|transmission|
+------------+
|     unknown|
|   automatic|
|       other|
|      manual|
+------------+

4


In [34]:
null_count = df8.filter(col("drive").isNull()).count()
print("null records" ,null_count)

null_count = df8.filter(col("drive").isNotNull()).count()
print("not null records" ,null_count)

total_count = df8.select('drive').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df8.groupBy('drive').count()
print(value_counts.show(900))

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df8.select("drive").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346840
total count  346840
--------------------------------------
+-------+------+
|  drive| count|
+-------+------+
|unknown|103160|
|    fwd| 86537|
|    rwd| 48135|
|    4wd|109008|
+-------+------+

None
--------------------------------------
+-------+
|  drive|
+-------+
|unknown|
|    fwd|
|    rwd|
|    4wd|
+-------+

4


In [35]:
from pyspark.sql.functions import countDistinct

# Iterate over each column in the DataFrame
for col_name in df8.columns:
    # Count the distinct values in the column
    distinct_count = df8.select(countDistinct(col_name)).collect()[0][0]
    print(col_name, distinct_count)

region 51
price 15274
year 107
manufacturer 21
model 51
condition 7
cylinders 9
fuel 6
odometer 101980
title_status 7
transmission 4
drive 4
type 14
paint_color 13


In [36]:
df8.describe()

DataFrame[summary: string, region: string, price: string, year: string, manufacturer: string, model: string, condition: string, cylinders: string, fuel: string, odometer: string, title_status: string, transmission: string, drive: string, type: string, paint_color: string]

In [37]:
null_count = df8.filter(col("year").isNull()).count()
print("null records" ,null_count)

null_count = df8.filter(col("year").isNotNull()).count()
print("not null records" ,null_count)

total_count = df8.select('year').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df8.groupBy('year').count()
print(value_counts.show())

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df8.select("year").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346840
total count  346840
--------------------------------------
+----+-----+
|year|count|
+----+-----+
|1953|   55|
|1957|  117|
|1987|  393|
|1956|   98|
|2016|24145|
|1936|   21|
|2020|17024|
|2012|18901|
|1958|   49|
|1943|    1|
|1915|    1|
|1972|  298|
|1931|   26|
|1988|  410|
|1926|   10|
|1938|   14|
|2019|21961|
|2017|30249|
|1932|   46|
|1977|  199|
+----+-----+
only showing top 20 rows

None
--------------------------------------
+----+
|year|
+----+
|1953|
|1957|
|1987|
|1956|
|2016|
|1936|
|2020|
|2012|
|1958|
|1972|
|1931|
|1988|
|1938|
|2019|
|1918|
|2017|
|1932|
|1977|
|2014|
|1971|
+----+
only showing top 20 rows

107


In [38]:
df8.printSchema()

root
 |-- region: string (nullable = false)
 |-- price: string (nullable = false)
 |-- year: string (nullable = false)
 |-- manufacturer: string (nullable = false)
 |-- model: string (nullable = false)
 |-- condition: string (nullable = false)
 |-- cylinders: string (nullable = false)
 |-- fuel: string (nullable = false)
 |-- odometer: string (nullable = false)
 |-- title_status: string (nullable = false)
 |-- transmission: string (nullable = false)
 |-- drive: string (nullable = false)
 |-- type: string (nullable = false)
 |-- paint_color: string (nullable = false)



In [39]:
# converting odometer to integer type:

# Convert data type from string to integer
df9 = df8.withColumn("year", col('year').cast("int"))

# Show the DataFrame schema to verify the changes
df9.printSchema()

root
 |-- region: string (nullable = false)
 |-- price: string (nullable = false)
 |-- year: integer (nullable = true)
 |-- manufacturer: string (nullable = false)
 |-- model: string (nullable = false)
 |-- condition: string (nullable = false)
 |-- cylinders: string (nullable = false)
 |-- fuel: string (nullable = false)
 |-- odometer: string (nullable = false)
 |-- title_status: string (nullable = false)
 |-- transmission: string (nullable = false)
 |-- drive: string (nullable = false)
 |-- type: string (nullable = false)
 |-- paint_color: string (nullable = false)



from pyspark.sql.functions import to_timestamp, year

# Assuming df is your Spark DataFrame with a column named 'timestamp_str' of string type
# Convert the string column to a timestamp column
df9 = df8.withColumn('year', to_timestamp(df8['year'], 'yyyy-MM-dd HH:mm:ss'))

# Extract the year from the timestamp column
df9 = df8.withColumn('year', year(df8['year']))


In [40]:
null_count = df9.filter(col("year").isNull()).count()
print("null records" ,null_count)

null_count = df9.filter(col("year").isNotNull()).count()
print("not null records" ,null_count)

total_count = df9.select('year').count()

print("total count ",total_count)

print("--------------------------------------")

value_counts = df9.groupBy('year').count()
print(value_counts.show())

print("--------------------------------------")

# Get unique records of a specific column
unique_records = df9.select("year").distinct()

# Show the unique records
unique_records.show()

# Count the number of unique records
count_unique_records = unique_records.count()
print(count_unique_records)

null records 0
not null records 346840
total count  346840
--------------------------------------
+----+-----+
|year|count|
+----+-----+
|1959|   55|
|1990|  494|
|1975|  134|
|1977|  199|
|1924|    6|
|2003| 5939|
|2007|11938|
|2018|31616|
|1974|  183|
|2015|25217|
|1927|   26|
|1955|  135|
|2006|10205|
|1978|  260|
|2022|   69|
|1961|   50|
|2013|24553|
|1942|    6|
|1939|   32|
|1952|   65|
+----+-----+
only showing top 20 rows

None
--------------------------------------
+----+
|year|
+----+
|1959|
|1990|
|1975|
|1977|
|1924|
|2003|
|2007|
|2018|
|1974|
|2015|
|1927|
|1955|
|2006|
|1978|
|1961|
|2013|
|1942|
|1939|
|1922|
|1952|
+----+
only showing top 20 rows

107


# odometer :

In [41]:
# converting odometer to integer type:

# Convert data type from string to integer
df10 = df9.withColumn("odometer", col('odometer').cast("int"))

# Show the DataFrame schema to verify the changes
df10.printSchema()

root
 |-- region: string (nullable = false)
 |-- price: string (nullable = false)
 |-- year: integer (nullable = true)
 |-- manufacturer: string (nullable = false)
 |-- model: string (nullable = false)
 |-- condition: string (nullable = false)
 |-- cylinders: string (nullable = false)
 |-- fuel: string (nullable = false)
 |-- odometer: integer (nullable = true)
 |-- title_status: string (nullable = false)
 |-- transmission: string (nullable = false)
 |-- drive: string (nullable = false)
 |-- type: string (nullable = false)
 |-- paint_color: string (nullable = false)



# PRICE :

In [42]:
# converting odometer to integer type:

# Convert data type from string to integer
df11 = df10.withColumn("price", col('price').cast("int"))

# Show the DataFrame schema to verify the changes
df11.printSchema()

root
 |-- region: string (nullable = false)
 |-- price: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- manufacturer: string (nullable = false)
 |-- model: string (nullable = false)
 |-- condition: string (nullable = false)
 |-- cylinders: string (nullable = false)
 |-- fuel: string (nullable = false)
 |-- odometer: integer (nullable = true)
 |-- title_status: string (nullable = false)
 |-- transmission: string (nullable = false)
 |-- drive: string (nullable = false)
 |-- type: string (nullable = false)
 |-- paint_color: string (nullable = false)



In [43]:
df11.show(10)

+------+-----+----+------------+--------------+---------+-----------+-------+--------+------------+------------+-------+-------+-----------+
|region|price|year|manufacturer|         model|condition|  cylinders|   fuel|odometer|title_status|transmission|  drive|   type|paint_color|
+------+-----+----+------------+--------------+---------+-----------+-------+--------+------------+------------+-------+-------+-----------+
|others|39590|2020|        ford|        others|     good|8 cylinders|    gas|    1419|       clean|       other|    rwd|  coupe|      white|
|others| 3850|2010|  volkswagen|         jetta|  unknown|    unknown|    gas|  178000|       clean|   automatic|unknown|unknown|    unknown|
|others|45950|2014|      others|        others|excellent|8 cylinders| diesel|  154289|       clean|   automatic|    4wd|  truck|      black|
|others|33990|2016|   chevrolet|silverado 1500|  unknown|8 cylinders|unknown|  117895|       clean|   automatic|    4wd| pickup|      white|
|others| 5000

In [44]:
from pyspark.sql.functions import countDistinct

# Iterate over each column in the DataFrame
for col_name in df11.columns:
    # Count the distinct values in the column
    distinct_count = df11.select(countDistinct(col_name)).collect()[0][0]
    print(col_name, distinct_count)

region 51
price 15272
year 107
manufacturer 21
model 51
condition 7
cylinders 9
fuel 6
odometer 101980
title_status 7
transmission 4
drive 4
type 14
paint_color 13


Our dataset have lots of outliers. Outliers reduces the accuracy of a model. We have to detect outliers and remove them.

# for price quartiles :

In [45]:
df11.count()

346840

In [46]:
# Calculate quartiles
price_percentiles = df11.approxQuantile("price", [0.15, 0.75], 0.01)  # Use a low relative error for better accuracy
price_percentile15 = price_percentiles[0]
price_percentile75 = price_percentiles[1]

# Calculate IQR and upper/lower limits
price_iqr = price_percentile75 - price_percentile15
price_upper_limit = price_percentile75 + 1.5 * price_iqr
price_lower_limit = price_percentile15

# Filter DataFrame based on limits
df12 = df11.filter((col("price") < price_upper_limit) & (col("price") > price_lower_limit))

In [47]:
df12.count()

289883

# for odometer quartiles :

In [48]:
# Calculate percentiles
odometer_percentiles = df12.approxQuantile("odometer", [0.05, 0.25, 0.75], 0.01)  # Use a low relative error for better accuracy
odometer_percentile05 = odometer_percentiles[0]
odometer_percentile25 = odometer_percentiles[1]
odometer_percentile75 = odometer_percentiles[2]

# Calculate IQR and upper/lower limits
odometer_iqr = odometer_percentile75 - odometer_percentile25
odometer_upper_limit = odometer_percentile75 + 1.5 * odometer_iqr
odometer_lower_limit = odometer_percentile05

# Filter DataFrame based on limits
df13 = df12.filter((col("odometer") < odometer_upper_limit) & (col("odometer") > odometer_lower_limit))

In [49]:
df13.count()

272791

# Feature Engineering :

In [50]:
import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
#pip install matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.


In [51]:
new_df = df13.select("price","year").toPandas()

# Set up plot parameters
plt.rcParams["figure.autolayout"] = True
sns.set_palette('summer_r')
plt.figure(figsize=(12, 9))

# Plot
sns.barplot(y=new_df['price'], x=new_df['year'])
plt.title("Price VS Year", fontsize=20, fontweight='bold')
plt.xlabel('Year', fontsize=20)
plt.xticks(rotation=60)
plt.ylabel('Price', fontsize=20)
plt.tight_layout()
plt.show()

Py4JError: An error occurred while calling o1208.pandasStructHandlingMode. Trace:
py4j.Py4JException: Method pandasStructHandlingMode([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:745)



We can see the car data before 1996 is quiet irrelevant with our current times data. So we are extracting all the car data available after 1996

In [52]:
# Filter DataFrame based on the condition
df14 = df13.where(df13['year'] > 1996)

In [53]:
from pyspark.sql.functions import countDistinct

# Iterate over each column in the DataFrame
for col_name in df14.columns:
    # Count the distinct values in the column
    distinct_count = df14.select(countDistinct(col_name)).collect()[0][0]
    print(col_name, distinct_count)

region 51
price 12648
year 26
manufacturer 21
model 51
condition 7
cylinders 9
fuel 6
odometer 83924
title_status 7
transmission 4
drive 4
type 14
paint_color 13


In [54]:
df14.count()

264038

In [55]:
# Calculate car age and add it as a new column
df15 = df14.withColumn('car_age', 2022 - col('year'))

# Drop the 'year' column
df16 = df15.drop('year')

# Describe the DataFrame to get summary statistics
df16.describe().show()

+-------+--------------+------------------+------------+------------------+---------+------------+-------+------------------+------------+------------+-------+------+-----------+-----------------+
|summary|        region|             price|manufacturer|             model|condition|   cylinders|   fuel|          odometer|title_status|transmission|  drive|  type|paint_color|          car_age|
+-------+--------------+------------------+------------+------------------+---------+------------+-------+------------------+------------+------------+-------+------+-----------+-----------------+
|  count|        264038|            264038|      264038|            264038|   264038|      264038| 264038|            264038|      264038|      264038| 264038|264038|     264038|           264038|
|   mean|          null|19676.750384414365|        null|2158.5919757759275|     null|        null|   null| 90331.25469440005|        null|        null|   null|  null|       null| 9.06306289246245|
| stddev|      

In [56]:
df16.printSchema()

root
 |-- region: string (nullable = false)
 |-- price: integer (nullable = true)
 |-- manufacturer: string (nullable = false)
 |-- model: string (nullable = false)
 |-- condition: string (nullable = false)
 |-- cylinders: string (nullable = false)
 |-- fuel: string (nullable = false)
 |-- odometer: integer (nullable = true)
 |-- title_status: string (nullable = false)
 |-- transmission: string (nullable = false)
 |-- drive: string (nullable = false)
 |-- type: string (nullable = false)
 |-- paint_color: string (nullable = false)
 |-- car_age: integer (nullable = true)



In [57]:
df16.count()

264038

### MODELS

In [79]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler,StandardScaler
from pyspark.ml.regression import LinearRegression,RandomForestRegressor,DecisionTreeRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [59]:
train, test = df16.randomSplit([0.8, 0.2])

In [60]:
numerical=["odometer","car_age"]

In [61]:
numerical_vector_assembler = VectorAssembler(inputCols=numerical,
                                             outputCol='numerical_feature_vector')

train = numerical_vector_assembler.transform(train)
test = numerical_vector_assembler.transform(test)

In [62]:
scaler = StandardScaler(inputCol='numerical_feature_vector',
                        outputCol='scaled_numerical_feature_vector',
                        withStd=True, withMean=True)

scaler = scaler.fit(train)

train = scaler.transform(train)
test = scaler.transform(test)

In [63]:
indexer = StringIndexer(inputCols=['manufacturer','model','condition','cylinders','fuel','title_status','transmission','drive','type','paint_color','region'],
                        outputCols=['manufacturer_index','m_i','co_i','cy_i','f_i','ts_i','tr_i','d_i','ty_i','p_i','r_i'],handleInvalid="keep")

indexer = indexer.fit(train)
train = indexer.transform(train)
test = indexer.transform(test)

In [64]:
one_hot_encoder = OneHotEncoder(inputCols=['manufacturer_index','m_i','co_i','cy_i','f_i','ts_i','tr_i','d_i','ty_i','p_i','r_i'],
                                outputCols=['manufacturer_index_h','m_i_h','co_i_h','cy_i_h','f_i_h','ts_i_h','tr_i_h','d_i_h','ty_i_h','p_i_h','r_i_h'])

one_hot_encoder = one_hot_encoder.fit(train)

train = one_hot_encoder.transform(train)
test = one_hot_encoder.transform(test)

In [65]:
assembler = VectorAssembler(inputCols=['scaled_numerical_feature_vector',
                                       'manufacturer_index_h','m_i_h','co_i_h','cy_i_h','f_i_h','ts_i_h','tr_i_h','d_i_h','ty_i_h','p_i_h','r_i_h'],
                            outputCol='final_feature_vector')

train = assembler.transform(train)
test = assembler.transform(test)

### linear regression

In [66]:
lr = LinearRegression(featuresCol='final_feature_vector',
                      labelCol='price')

In [67]:
lr=lr.fit(train)

In [68]:
#pred_train_df = lr.transform(train).withColumnRenamed('prediction',
                                                      #'predicted_vehicle_value')

#pred_train_df.show(5)

In [73]:
pred_test_df = lr.transform(test).withColumnRenamed('prediction', 'predicted_vehicle_value')

In [69]:
result=lr.evaluate(test)

In [70]:
print(result.r2)

0.7439798435931946


In [71]:
unlabeled_data=test.select("final_feature_vector")

In [72]:
predictions=lr.transform(unlabeled_data)

In [73]:
predictions.show()

+--------------------+-------------------+
|final_feature_vector|         prediction|
+--------------------+-------------------+
|(189,[0,1,6,33,75...|  813.5642205286167|
|(189,[0,1,13,23,7...| -6567.087376363495|
|(189,[0,1,6,23,74...|  3049.477467715249|
|(189,[0,1,16,23,7...|  1423.821507894143|
|(189,[0,1,6,32,75...| 2413.9466519904163|
|(189,[0,1,4,23,76...| 11238.295942869725|
|(189,[0,1,6,32,75...| 12713.459846554571|
|(189,[0,1,12,23,7...|  4136.175756756533|
|(189,[0,1,13,23,7...|-2137.7855483546955|
|(189,[0,1,5,29,76...| 1575.9921574458494|
|(189,[0,1,6,23,76...| 12763.484107470536|
|(189,[0,1,3,23,76...| 12415.112055633128|
|(189,[0,1,6,23,76...|  7551.194138699946|
|(189,[0,1,10,23,7...| 16671.914135681014|
|(189,[0,1,3,23,76...|  16805.15914851604|
|(189,[0,1,4,23,76...| 14778.585814796577|
|(189,[0,1,10,23,7...|  7993.307520564165|
|(189,[0,1,3,23,76...| 14798.339473108019|
|(189,[0,1,5,23,76...| 14578.241391875963|
|(189,[0,1,5,23,76...| 13956.766616853034|
+----------

### random forest

In [74]:
rf = RandomForestRegressor(featuresCol='final_feature_vector',
                      labelCol='price')

In [75]:
rf = rf.fit(train)

In [87]:
predictions = rf.transform(test)

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")

# Evaluate the model
r2 = evaluator.evaluate(predictions)
mse = evaluator.evaluate(predictions, {evaluator.metricName: "mse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})

# Print the evaluation metrics
print("R2 on test data = %g" % r2)
print("Mean squared error:", mse)
print("Mean absolute error:", mae)
print("Root mean squared error:", rmse)

# Show predictions
predictions.select("final_feature_vector", "price", "prediction").show()

R2 on test data = 0.690532
Mean squared error: 46713591.48371444
Mean absolute error: 5172.064741192836
Root mean squared error: 6834.734192616012
+--------------------+-----+------------------+
|final_feature_vector|price|        prediction|
+--------------------+-----+------------------+
|(189,[0,1,6,33,75...| 4000| 7769.644944797677|
|(189,[0,1,13,23,7...| 4000|10060.292738028387|
|(189,[0,1,6,23,74...| 4299|10145.533259097834|
|(189,[0,1,16,23,7...| 4900|10145.533259097834|
|(189,[0,1,6,32,75...| 5900| 8963.416251666618|
|(189,[0,1,4,23,76...| 6299| 10647.48307463534|
|(189,[0,1,6,32,75...| 6500|10145.533259097834|
|(189,[0,1,12,23,7...| 7000|12146.756368367842|
|(189,[0,1,13,23,7...| 7500| 9362.404266808418|
|(189,[0,1,5,29,76...| 7888|  8235.33122560976|
|(189,[0,1,6,23,76...| 8200|17534.784983943275|
|(189,[0,1,3,23,76...| 9500|19394.953044085763|
|(189,[0,1,6,23,76...| 9800|11743.376511348582|
|(189,[0,1,10,23,7...| 9888|10927.121909063644|
|(189,[0,1,3,23,76...| 9988|10822.644

### decision tree

In [88]:
# Define Decision Tree Regressor
dt = DecisionTreeRegressor(featuresCol='final_feature_vector',
                      labelCol='price')

# Train the model
dt = dt.fit(train)

# Make predictions
predictions = dt.transform(test)

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
mse = evaluator.evaluate(predictions, {evaluator.metricName: "mse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})

# Print the evaluation metrics
print("R2 on test data = %g" % r2)
print("Mean squared error:", mse)
print("Mean absolute error:", mae)
print("Root mean squared error:", rmse)

# Show predictions
predictions.select("final_feature_vector", "price", "prediction").show()

R2 on test data = 0.645953
Mean squared error: 53442766.04141012
Mean absolute error: 5413.173757862375
Root mean squared error: 7310.4559393658965
+--------------------+-----+------------------+
|final_feature_vector|price|        prediction|
+--------------------+-----+------------------+
|(189,[0,1,6,33,75...| 4000|7779.1648925013105|
|(189,[0,1,13,23,7...| 4000|7779.1648925013105|
|(189,[0,1,6,23,74...| 4299|7779.1648925013105|
|(189,[0,1,16,23,7...| 4900|7779.1648925013105|
|(189,[0,1,6,32,75...| 5900|7779.1648925013105|
|(189,[0,1,4,23,76...| 6299|7779.1648925013105|
|(189,[0,1,6,32,75...| 6500|7779.1648925013105|
|(189,[0,1,12,23,7...| 7000|  8278.15276145711|
|(189,[0,1,13,23,7...| 7500|7779.1648925013105|
|(189,[0,1,5,29,76...| 7888|7779.1648925013105|
|(189,[0,1,6,23,76...| 8200|15401.746406192407|
|(189,[0,1,3,23,76...| 9500|19391.121109686443|
|(189,[0,1,6,23,76...| 9800|  8278.15276145711|
|(189,[0,1,10,23,7...| 9888|7779.1648925013105|
|(189,[0,1,3,23,76...| 9988| 10855.0

### ridge regression

In [98]:
# Define Ridge Regression model
ridge = LinearRegression(featuresCol="final_feature_vector", labelCol="price", elasticNetParam=0.0, regParam=0.1) # regParam is the regularization parameter for Ridge Regression

# Train the model
ridge_model = ridge.fit(train)

# Make predictions
predictions = ridge_model.transform(test)

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
mse = evaluator.evaluate(predictions, {evaluator.metricName: "mse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
print("R2 on test data = %g" % r2)
print("Mean squared error:", mse)
print("Mean absolute error:", mae)
print("Root mean squared error:", rmse)
# Show predictions
predictions.select("final_feature_vector", "price", "prediction").show()

R2 on test data = 0.74398
Mean squared error: 38645782.02751786
Mean absolute error: 4630.695133415567
Root mean squared error: 6216.573173985637
+--------------------+-----+-------------------+
|final_feature_vector|price|         prediction|
+--------------------+-----+-------------------+
|(189,[0,1,6,33,75...| 4000|  813.6450525851214|
|(189,[0,1,13,23,7...| 4000| -6566.949417342836|
|(189,[0,1,6,23,74...| 4299| 3049.5833724085714|
|(189,[0,1,16,23,7...| 4900|  1423.947617681697|
|(189,[0,1,6,32,75...| 5900|  2414.008285201344|
|(189,[0,1,4,23,76...| 6299| 11238.356954198518|
|(189,[0,1,6,32,75...| 6500| 12713.436937324019|
|(189,[0,1,12,23,7...| 7000|  4136.299271966451|
|(189,[0,1,13,23,7...| 7500|-2137.6710813212812|
|(189,[0,1,5,29,76...| 7888|  1576.104174357115|
|(189,[0,1,6,23,76...| 8200|  12763.45509697066|
|(189,[0,1,3,23,76...| 9500| 12415.140145254307|
|(189,[0,1,6,23,76...| 9800|  7551.233221046634|
|(189,[0,1,10,23,7...| 9888| 16671.942701103675|
|(189,[0,1,3,23,76...

### lasso regression

In [101]:
# Define Ridge Regression model
lasso = LinearRegression(featuresCol="final_feature_vector", labelCol="price", elasticNetParam=1.0, regParam=0.1) # regParam is the regularization parameter for Ridge Regression

# Train the model
las_model = lasso.fit(train)

# Make predictions
predictions = las_model.transform(test)

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
mse = evaluator.evaluate(predictions, {evaluator.metricName: "mse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
print("R2 on test data = %g" % r2)
print("Mean squared error:", mse)
print("Mean absolute error:", mae)
print("Root mean squared error:", rmse)
# Show predictions
predictions.select("final_feature_vector", "price", "prediction").show()

R2 on test data = 0.74398
Mean squared error: 38645781.511699185
Mean absolute error: 4630.679619225646
Root mean squared error: 6216.573132498256
+--------------------+-----+------------------+
|final_feature_vector|price|        prediction|
+--------------------+-----+------------------+
|(189,[0,1,6,33,75...| 4000| 813.5681342342177|
|(189,[0,1,13,23,7...| 4000|-6564.296193667469|
|(189,[0,1,6,23,74...| 4299|3049.9660344017066|
|(189,[0,1,16,23,7...| 4900| 1423.240126815097|
|(189,[0,1,6,32,75...| 5900|  2412.57145013678|
|(189,[0,1,4,23,76...| 6299|11237.485516999874|
|(189,[0,1,6,32,75...| 6500|12712.415571844558|
|(189,[0,1,12,23,7...| 7000| 4136.071202736841|
|(189,[0,1,13,23,7...| 7500| -2137.42677434046|
|(189,[0,1,5,29,76...| 7888|1576.4124258168886|
|(189,[0,1,6,23,76...| 8200|12765.295851514074|
|(189,[0,1,3,23,76...| 9500|12415.540525912736|
|(189,[0,1,6,23,76...| 9800| 7550.973071354691|
|(189,[0,1,10,23,7...| 9888| 16669.55996150455|
|(189,[0,1,3,23,76...| 9988| 16804.48

### bagging regressor

In [130]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.sql.functions import rand,col
from functools import reduce

In [ ]:
# Define the number of base estimators (decision trees)
n_estimators = 50

# Initialize an empty DataFrame to store the predictions
predictions = None

# Train n_estimators decision trees on bootstrap samples
for i in range(n_estimators):
    # Sample with replacement to create a bootstrap sample
    bootstrap_sample = train.orderBy(rand()).limit(train.count())
    
    # Train a decision tree on the bootstrap sample
    dt = DecisionTreeRegressor(maxDepth=20)
    dt_model = dt.fit(bootstrap_sample)
    
    # Make predictions on the test set
    if predictions is None:
        predictions = dt_model.transform(X_test_tnf).select("prediction").withColumnRenamed("prediction", f"prediction_{i}")
    else:
        predictions = predictions.join(dt_model.transform(X_test_tnf).select("prediction").withColumnRenamed("prediction", f"prediction_{i}"), how="outer")

# Average the predictions from all the decision trees
predictions = predictions.withColumn("avg_prediction", sum(predictions[col] for col in predictions.columns) / n_estimators)

# Show the averaged predictions
predictions.show()

In [128]:
# Combine feature columns into a single vector column
#vector_assembler = VectorAssembler(inputCols=train.columns, outputCol="features")
#X_train_tnf = vector_assembler.transform(train)
#X_test_tnf = vector_assembler.transform(test)

'''
# Define the number of base estimators (decision trees)
n_estimators = 50

# Initialize an empty DataFrame to store the predictions
predictions = None

# Train n_estimators decision trees on bootstrap samples
for i in range(n_estimators):
    # Sample with replacement to create a bootstrap sample
    bootstrap_sample = train.orderBy(rand()).limit(train.count())
    
    # Train a decision tree on the bootstrap sample
    dt = DecisionTreeRegressor()
    dt_model = dt.fit(bootstrap_sample)
    
    # Make predictions on the test set
    if predictions is None:
        predictions = dt_model.transform(test).select("prediction").withColumnRenamed("prediction", f"prediction_{i}")
    else:
        predictions = predictions.join(dt_model.transform(test).select("prediction").withColumnRenamed("prediction", f"prediction_{i}"), how="outer")

# Average the predictions from all the decision trees
predictions = predictions.withColumn("avg_prediction", sum(predictions[col] for col in predictions.columns) / n_estimators)
'''
num_models = 5

# Train individual decision tree models on different bootstrap samples of the data
models = []
for i in range(num_models):
    # Sample with replacement to create a bootstrap sample
    bootstrap_sample = train.sample(True, 1.0, seed=i)
    
    # Train a DecisionTreeRegressor model on the bootstrap sample
    dt = DecisionTreeRegressor(featuresCol="final_feature_vector", labelCol="price")
    model = dt.fit(bootstrap_sample)
    
    models.append(model)

# Make predictions using each model
predictions = [model.transform(train) for model in models]

#Combine predictions into a single DataFrame
combined_predictions = reduce(DataFrame.unionAll, predictions)
#combined_predictions = predictions[0]
#for pred in predictions[1:]:
#    combined_predictions = combined_predictions.union(pred)

# Group by the original data row and average the predictions
avg_predictions = combined_predictions.groupBy("final_feature_vector").agg({"prediction": "avg"}).withColumnRenamed("avg(prediction)", "avg_prediction")

# Show the averaged predictions
avg_predictions.show()

#for df in predictions:
    #df.show() 

# Show the averaged predictions
#predictions.show()# Evaluate the model
evaluator = RegressionEvaluator(metricName="r2")
r2_score = evaluator.evaluate(avg_predictions)
mse = evaluator.evaluate(avg_predictions, {evaluator.metricName: "mse"})
mae = evaluator.evaluate(avg_predictions, {evaluator.metricName: "mae"})
rmse = evaluator.evaluate(avg_predictions, {evaluator.metricName: "rmse"})

# Print the evaluation metrics
print("R2 score:", r2_score)
print("Mean squared error:", mse)
print("Mean absolute error:", mae)
print("Root mean squared error:", rmse)

+--------------------+------------------+
|final_feature_vector|    avg_prediction|
+--------------------+------------------+
|(189,[0,1,2,28,75...| 7775.094758240372|
|(189,[0,1,6,23,74...| 7775.094758240372|
|(189,[0,1,17,23,7...| 7775.094758240372|
|(189,[0,1,3,23,76...|10834.868785126786|
|(189,[0,1,17,23,7...|15443.967254135863|
|(189,[0,1,4,23,76...|20133.453758237498|
|(189,[0,1,3,23,76...|21559.847957479382|
|(189,[0,1,12,23,7...|32457.199622432865|
|(189,[0,1,3,23,75...|24551.034497952285|
|(189,[0,1,6,32,74...| 7775.094758240372|
|(189,[0,1,2,36,75...|25468.475195431925|
|(189,[0,1,2,36,74...|32457.199622432865|
|(189,[0,1,2,24,74...|32457.199622432865|
|(189,[0,1,2,23,76...| 9453.516347365618|
|(189,[0,1,3,23,75...|32457.199622432865|
|(189,[0,1,16,23,7...|12399.573280763845|
|(189,[0,1,2,41,76...|11937.977726974419|
|(189,[0,1,7,23,74...|13845.011563649581|
|(189,[0,1,8,23,76...|25285.111156144423|
|(189,[0,1,3,23,74...|32457.199622432865|
+--------------------+------------

IllegalArgumentException: prediction does not exist. Available: final_feature_vector, avg_prediction

### xgboost

In [132]:
from sparkxgb import XGBoostRegressor

In [133]:
xgb = XGBoostRegressor(
    featuresCol="final_feature_vector",
    labelCol="price",
    predictionCol="prediction"
)

# Train the model
xgb_model = xgb.fit(train)

# Make predictions
predictions = xgb_model.transform(test)

# Show predictions
predictions.select("final_feature_vector", "price", "prediction").show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Ajinkya\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Ajinkya\AppData\Local\Temp\ipykernel_13336\662249834.py", line 1, in <module>
    xgb = XGBoostRegressor(
          ^^^^^^^^^^^^^^^^^
  File "C:\Users\Ajinkya\anaconda3\Lib\site-packages\pyspark\__init__.py", line 139, in wrapper
  File "C:\Users\Ajinkya\anaconda3\Lib\site-packages\sparkxgb\xgboost.py", line 232, in __init__
    super(XGBoostRegressor, self).__init__(classname="ml.dmlc.xgboost4j.scala.spark.XGBoostRegressor")
  File "C:\Users\Ajinkya\anaconda3\Lib\site-packages\sparkxgb\common.py", line 68, in __init__
    self._java_obj = self._new_java_obj(classname, self.uid)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Ajinkya\anaconda3\Lib\site-packages\pyspark\ml\wrapper.py", line 86, in _new_java_obj
    Example primitive Java classe